In [2]:
# Utility Libraries
import numpy as np
import pandas as pd
import sys
import time
import random
from scipy import linalg
import cv2
from PIL import Image
from PIL.Image import fromarray
import os

# Pytorch relevant librariesa
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import cat
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms, models
from torchvision.models import inception_v3
from torch.utils.data import random_split

# Plotting Libraries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.manifold import TSNE

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
with open('/kaggle/input/iam-lines/IAM/gt_test.txt') as f:
    contents = f.readlines()

lines = [line.strip() for line in contents]

In [5]:
file_names = []
texts = []

for line in lines:
    file_name = line.split('\t')[0]
    text = line.split('\t')[-1]
    
    file_names.append(file_name)
    texts.append(text)

In [6]:
lines_data = pd.DataFrame()
lines_data['file_name'] = file_names
lines_data['text'] = texts

In [7]:
lines_data

,file_name,text
0,c04-110-00.jpg,Become a success with a disc and hey presto ! ...
1,c04-110-01.jpg,"assuredness "" Bella Bella Marie "" ( Parlophone..."
2,c04-110-02.jpg,I don't think he will storm the charts with th...
3,c04-110-03.jpg,"CHRIS CHARLES , 39 , who lives in Stockton-on-..."
4,c04-116-00.jpg,He is also a director of a couple of garages ....
...,...,...
2910,r03-053-04.jpg,"just dusty-grey but muddy , slimy even ."
2911,r03-053-05.jpg,"His digressions too , seemed to have no other"
2912,r03-053-06.jpg,purpose than the throwing of dust in his
2913,r03-053-07.jpg,"client's eyes , the dust of fake security , of"


In [8]:
char_list = "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz " 

def encode_to_labels(text, max_word_length=85):
    dig_list = []
    for index in range(max_word_length):
        char = 79
        if index < len(text):
            char = char_list.index(text[index])
        dig_list.append(char) 
        
    return dig_list

In [9]:
class IAMDataset(Dataset):
    def __init__(self, data, root_dir, transform=None):
        self.data = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        file_name = self.data['file_name'][idx]
        text = self.data['text'][idx]

        file_name = file_name
        file_path = f"{self.root_dir}/{file_name}"

        # Load image
        try:
            image = Image.open(file_path).convert("RGB")
        except:
            image = torch.zeros((1, 30, 900))
            label = torch.zeros((85))
            return image, label


        if self.transform:
            image = self.transform(image)
            
        label = torch.tensor(encode_to_labels(text))

        return image, torch.tensor(label)

In [10]:
transform = transforms.Compose([
    transforms.Grayscale(),   
    transforms.Resize((30, 900)),
    transforms.ToTensor()
])

In [11]:
dataset = IAMDataset(data=lines_data, root_dir='/kaggle/input/iam-lines/IAM/image', transform=transform)

In [12]:
batch_size = 1

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [13]:
def visualize_image(dataloader):
    for image, labels in dataloader:
        break
        
    image = image[0]  # first image
    plt.figure(figsize=(4, 16))
    plt.imshow(image.permute(1, 2, 0), cmap='gray')
    label = ''.join([char_list[index] for index in labels[0] if index != 79])
    print(label)
    plt.axis('off')
    plt.show()

In [14]:
visualize_image(train_loader)

Gay , a little puzzled , went through


In [15]:
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()

        # Load pre-trained ResNet34 model
        self.resnet = models.resnet34(pretrained=True)

        # Modify the first convolutional layer to accommodate grayscale images
        conv1_weight = self.resnet.conv1.weight.data
        conv1_weight = conv1_weight.mean(dim=1, keepdim=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.conv1.weight.data = conv1_weight

        # Remove the last fully connected layer of ResNet34
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])

        # Freeze the ResNet layers so they won't be trained
        for param in self.resnet.parameters():
            param.requires_grad = False

    def forward(self, x):
        # Forward pass through ResNet layers
        x = self.resnet(x)
        return x

In [20]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(Encoder, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(self.input_dim, self.hidden_dim, num_layers=self.num_layers)
              
    def forward(self, x):
        outputs, hidden = self.gru(x)
        return outputs, hidden

In [21]:
class Decoder(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_layers):
        super(Decoder, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, self.hidden_dim, num_layers=self.num_layers)
        self.out = nn.Linear(self.hidden_dim, output_dim)

    def forward(self, x, hidden):
        x = x.view(1, -1)
        output, hidden = self.gru(x, hidden)   
        prediction = self.out(output[0])

        return prediction, hidden

In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, enc_input_dim, dec_input_dim, hidden_dim, target_length=100, num_chars=80):
        super(Seq2Seq, self).__init__()
        self.target_length = target_length
        self.num_chars = num_chars
        
        self.encoder = Encoder(
            input_dim=enc_input_dim,
            hidden_dim=hidden_dim,
            num_layers=1
        ).to(device)
        
        self.decoder = Decoder(
            input_dim=dec_input_dim,
            output_dim=dec_input_dim,
            hidden_dim=hidden_dim,
            num_layers=1
        ).to(device)
        
    def forward(self, features):
        seq_length = features.shape[0]
        
        for i in range(seq_length):
            enc_output, enc_hidden = self.encoder(features[i])
            
        outputs = torch.zeros(self.target_length, self.num_chars).to(device)
        
        dec_hidden = enc_hidden
        dec_input = torch.zeros(1, num_chars).to(device)

        for i in range(self.target_length):
            decoder_output, dec_hidden = self.decoder(dec_input, dec_hidden)
            outputs[i] = decoder_output
            dec_input = decoder_output
            
        return outputs

In [23]:
class HandwrittenLineRecognizer(nn.Module):
    def __init__(self, feature_extractor, seq2seq):
        super(HandwrittenLineRecognizer, self).__init__()
        self.feature_extractor = feature_extractor.to(device)
        self.seq2seq = seq2seq.to(device)
        
    def forward(self, line):
        features = self.feature_extractor(line)
        features = features.squeeze(2).permute(2, 0, 1)
        output = self.seq2seq(features).unsqueeze(0)
        output = output.permute(0, 2, 1)
        
        return output

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
enc_input_dim = 512
dec_input_dim = 80
hidden_dim = 256
target_length = 85
num_chars = 80

feature_extractor = FeatureExtractor()
seq2seq = Seq2Seq(enc_input_dim, dec_input_dim, hidden_dim, target_length, num_chars)

In [32]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=(2, 1))
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.batch_norm5 = nn.BatchNorm2d(512)
        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.batch_norm6 = nn.BatchNorm2d(512)
        self.pool6 = nn.MaxPool2d(kernel_size=(2, 2))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool4(x)
        x = F.relu(self.conv5(x))
        x = self.batch_norm5(x)
        x = F.relu(self.conv6(x))
        x = self.batch_norm6(x)
        x = self.pool6(x)
        x = x.squeeze(2)  # Squeeze the height dimension
        x = x.permute(2, 0, 1)  # Reshape for RNN input
        return x

In [33]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(1), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(1), self.hidden_size).to(device)
        output, _ = self.lstm(x, (h0, c0))
        output = self.fc(output)
        return output

In [48]:
class HandwritingRecognitionModel(nn.Module):
    def __init__(self, num_classes):
        super(HandwritingRecognitionModel, self).__init__()
        self.cnn = CNN()
        self.rnn = RNN(512, 256, 2, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        x = self.rnn(x)
        return x

In [49]:
num_classes = 80  # 78 characters + 1 for blank
num_epochs = 10
model = HandwritingRecognitionModel(num_classes)

# Loss function
criterion = nn.CTCLoss(blank=79, zero_infinity=True)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [51]:
T = 112
C = 80
S = 85
S_min = 84

loss_trajectory = []
model = HandwritingRecognitionModel(num_classes).to(device)
for epoch in range(10):
    losses = []
    num_batches = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        N = images.shape[0]
        optimizer.zero_grad()
        outputs = model(images)
        input_lengths = torch.full(size=(N,), fill_value=T, dtype=torch.long)
        target_lengths = torch.randint(low=S_min, high=S, size=(N,), dtype=torch.long)
        loss = criterion(outputs, labels, input_lengths, target_lengths)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        num_batches += 1
        
        if num_batches % 500 == 0:
            print(f'Epoch {epoch + 1}, Batch : {num_batches}, Loss : {loss.item()}')
        loss_trajectory.append(np.round(np.mean(losses), 3))
    print(f"For Epoch {epoch + 1} : Loss : {np.round(np.mean(losses), 3)}")
    print()

Epoch 1, Batch : 500, Loss : -0.05242469161748886
Epoch 1, Batch : 1000, Loss : 0.0
Epoch 1, Batch : 1500, Loss : 0.050859421491622925
Epoch 1, Batch : 2000, Loss : 0.0
Epoch 1, Batch : 2500, Loss : 0.0
For Epoch 1 : Loss : -0.019

Epoch 2, Batch : 500, Loss : 0.0
Epoch 2, Batch : 1000, Loss : 0.0
Epoch 2, Batch : 1500, Loss : 0.0
Epoch 2, Batch : 2000, Loss : 0.0
Epoch 2, Batch : 2500, Loss : 0.0
For Epoch 2 : Loss : -0.019

Epoch 3, Batch : 500, Loss : 0.0
Epoch 3, Batch : 1000, Loss : 0.0
Epoch 3, Batch : 1500, Loss : 0.0
Epoch 3, Batch : 2000, Loss : 0.0
Epoch 3, Batch : 2500, Loss : 0.0
For Epoch 3 : Loss : -0.019

Epoch 4, Batch : 500, Loss : 0.0
Epoch 4, Batch : 1000, Loss : 0.0
Epoch 4, Batch : 1500, Loss : -0.3164207935333252
Epoch 4, Batch : 2000, Loss : 0.0
Epoch 4, Batch : 2500, Loss : 0.0
For Epoch 4 : Loss : -0.019

Epoch 5, Batch : 500, Loss : 0.0
Epoch 5, Batch : 1000, Loss : 0.0036552411038428545
Epoch 5, Batch : 1500, Loss : 0.0
Epoch 5, Batch : 2000, Loss : 0.0
Epoch

In [52]:
def get_text(sequence):
    text = ""
    for i in sequence:
        i = i.item()
        if i != 79:
            text += char_list[i]
    return text

In [53]:
def ctc_greedy_decode_batch(logits, blank_index=0):
    """
    Greedy decoding for CTC output for a batch of sequences.
    
    Args:
    logits (np.array): Logits from the model, shape (T, batch_size, C),
                       where T is the sequence length, batch_size is the
                       batch size, and C is the number of characters.
    blank_index (int): Index of the blank symbol.
    
    Returns:
    decoded_sequences (list of lists): Decoded sequences of characters for each batch element.
    """
    T, batch_size, C = logits.shape
    decoded_sequences = []

    for b in range(batch_size):
        # Get the logits for the current batch element
        batch_logits = logits[:, b, :]
        
        # Get the index with maximum probability for each timestep
        best_path = np.argmax(batch_logits, axis=1)
        
        # Collapse consecutive duplicates and remove blank symbols
        decoded_sequence = [best_path[0]]
        for i in range(1, len(best_path)):
            if best_path[i] != blank_index and best_path[i] != best_path[i - 1]:
                decoded_sequence.append(best_path[i])
        
        decoded_sequences.append(decoded_sequence)

    output_text = []
    for sequence in decoded_sequences:
        output_text.append(get_text(sequence))
    
    return output_text

In [54]:
decoded_sequences = ctc_greedy_decode_batch(outputs.detach().cpu())

In [55]:
print(decoded_sequences)

['fMfMblWMlwlwbfblkfkbEbflflfkbMlbl6bEfbfbMlMlbklfbfbfMfEkwW']
